In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from influxdb import *
import datetime
from math import sqrt

In [2]:
INFLUX_DB_IP = '10.12.97.178'
INFLUX_DB_PORT = 8086
INFLUX_DB = 'demoDb'
TARGET_MEASUREMENT = 'EM_live'

con_obj = InfluxDBClient(host=INFLUX_DB_IP, port=INFLUX_DB_PORT, database=INFLUX_DB)
query = ('select "DeviceID", "EM_Power Factor", "EM_Active Power (kW)", "mean_volt", "Mean_THD", "EM_THD Voltage", "EM_Voltage Ph1-Ph2 (V)", "EM_Voltage Ph2-Ph3 (V)", "EM_Voltage Ph1-Ph3 (V)", "EM_Current Ph1 (A)", "EM_Current Ph2 (A)", "EM_Current Ph3 (A)", "mean_current", "time" from ' + TARGET_MEASUREMENT + ' where time > now() - 30m')
df = pd.DataFrame(con_obj.query(query).get_points())
df['time'] = df['time'].astype('datetime64[ns]')
df['time'] = df['time'] + datetime.timedelta(hours=5, minutes=30)

In [3]:
df.head()

,time,DeviceID,EM_Power Factor,EM_Active Power (kW),mean_volt,Mean_THD,EM_THD Voltage,EM_Voltage Ph1-Ph2 (V),EM_Voltage Ph2-Ph3 (V),EM_Voltage Ph1-Ph3 (V),EM_Current Ph1 (A),EM_Current Ph2 (A),EM_Current Ph3 (A),mean_current
0,2020-07-21 08:47:21.100369920,EM7,0.6,0.0,391.233333,0.000000,0.0,390.6,391.4,391.7,0.0,0.0,0.0,0.000000
1,2020-07-21 08:47:21.143343872,EM6,0.8,1050.7,393.533333,0.000000,0.0,392.5,393.8,394.3,1.9,1.9,1.9,1.900000
2,2020-07-21 08:47:21.230289920,EM23,1.0,60417.7,413.433333,0.000000,0.0,412.9,414.5,412.9,80.7,91.6,80.7,84.333333
3,2020-07-21 08:47:22.316519936,EM1,0.9,1298.8,422.500000,3.733333,1.9,423.5,423.1,420.9,1819.0,1819.4,1793.6,1810.666667
4,2020-07-21 08:47:22.360495104,EM10,0.8,55831.2,411.766667,0.000000,0.0,412.6,411.5,411.2,148.5,134.8,123.5,135.600000


In [4]:
len(df)

6214

In [5]:
class PowerFactorLoss_KPI():
    x = []

    def __init__(self, fluctuation_threshold):
        self.fluctuation_threshold = fluctuation_threshold
        
    def initialize(self, power):
        self.counter = 1
        self.average_power = power
        return
    
    def update_average_power(self, power):
        return (self.average_power * self.counter + power) / (self.counter + 1)
    
      
    def power_fluctuation(self, power):
        if abs(power - self.average_power) / self.average_power <= self.fluctuation_threshold:
            self.average_power = self.update_average_power(power)
            self.x.append(0)
        else:
            self.initialize(power)
            self.x.append(1)
        return self.x

    
    def check(self, df):
        for j in list(pd.unique(df['DeviceID'])):
            df_new = df[df['DeviceID'] == j] 
            df_new = df_new.reset_index(drop=True)
            self.initialize(df_new['EM_Active Power (kW)'].iloc[0])
            for i in range(0, len(df_new)):
                x = self.power_fluctuation(df_new['EM_Active Power (kW)'].iloc[i])
        return x 
    
    def categorize(self, df):
        df['cat'] = None
        cat = 0
        for i in range(len(df)):
            flag = df['flag'].iloc[i]
            if flag==0:
                df['cat'].iloc[i] = cat
                continue
            else:
                cat += 1
                df['cat'].iloc[i] = cat
        return df
    
    def kvah_loss_thd(self, df):
        df['THD_S'] = df['EM_Active Power (kW)'] / df['EM_Power Factor']
        df['THD_S1'] = df.apply(lambda x: (1/sqrt(1 + (x['EM_THD Voltage']*x['EM_THD Voltage']))) * (1/sqrt(1 + (x['Mean_THD']*x['Mean_THD']))),axis=1)
        df['kvah_loss_thd'] = df['THD_S']*df['time']*(1 - df['THD_S1'])
        return df
    
    def kvah_loss_pf(self, df):
        df['PF_S'] = df['EM_Active Power (kW)']/df['EM_Power Factor']
        df['PF_S1'] = df['EM_Active Power (kW)']/0.99
        df['kvah_loss_pf'] = df['time']*(df['PF_S'] - df['PF_S1'])
        return df

    def voltage_imbalance(self, df):
        df['Vmax'] = df[['EM_Voltage Ph1-Ph2 (V)', 'EM_Voltage Ph2-Ph3 (V)', 'EM_Voltage Ph1-Ph3 (V)']].max(axis = 1)
        df['Vmin'] = df[['EM_Voltage Ph1-Ph2 (V)', 'EM_Voltage Ph2-Ph3 (V)', 'EM_Voltage Ph1-Ph3 (V)']].min(axis = 1)
        x = df.groupby(['DeviceID', 'cat'])['Vmax'].max().reset_index()
        y = df.groupby(['DeviceID', 'cat'])['Vmin'].min().reset_index()
        df = df.merge(x, on = ['DeviceID', 'cat', 'Vmax'])
        df = df.merge(y, on = ['DeviceID', 'cat', 'Vmin'])
        df['voltage_imb'] = np.where((df['Vmax']-df['mean_volt']) > (df['mean_volt']-df['Vmin']),(df['Vmax']-df['mean_volt']),(df['mean_volt']-df['Vmin']))
        df['Vimb_per'] = (df['voltage_imb']/df['mean_volt'])*100
        return df
    
    def current_imbalance(self, df):
        df['Imax'] = df[['EM_Current Ph1 (A)', 'EM_Current Ph2 (A)', 'EM_Current Ph3 (A)']].max(axis = 1)
        df['Imin'] = df[['EM_Current Ph1 (A)', 'EM_Current Ph2 (A)', 'EM_Current Ph3 (A)']].min(axis = 1)
        a = df.groupby(['DeviceID', 'cat'])['Imax'].max().reset_index()
        b = df.groupby(['DeviceID', 'cat'])['Imin'].min().reset_index()
        df = df.merge(a, on = ['DeviceID', 'cat', 'Imax'])
        df = df.merge(b, on = ['DeviceID', 'cat', 'Imin'])
        df['current_imb'] = np.where((df['Imax']-df['mean_current']) > (df['mean_current']-df['Imin']),(df['Imax']-df['mean_current']),(df['mean_current']-df['Imin']))
        df['Iimb_per'] = (df['current_imb']/df['mean_current'])*100
        return df
    
    def time_diff(self, df):
        df = df.groupby(['DeviceID', 'cat'])['time'].apply(lambda y: (y.iloc[-1] - y.iloc[0]).total_seconds()/60).reset_index()
        return df
    
    def output(self, df):
        df['flag'] = self.check(df)
        df = self.categorize(df)
        t = self.time_diff(df)
        df = df.groupby(['DeviceID', 'cat']).agg(['mean']).reset_index()
        df = df.droplevel(1, axis=1)
        df = df.drop('flag', axis=1)
        df = df.merge(t , on = ['cat','DeviceID'])
        df = self.kvah_loss_thd(df)
        kvah_loss_thd = df.groupby('DeviceID')['kvah_loss_thd'].sum()
        df = self.kvah_loss_pf(df)
        kvah_loss_pf = df.groupby('DeviceID')['kvah_loss_pf'].sum()
        df = self.voltage_imbalance(df)
        Vimb_per = df.groupby('DeviceID')['Vimb_per'].mean()
        df = self.current_imbalance(df)
        Iimb_per = df.groupby('DeviceID')['Iimb_per'].mean()
        df_final = pd.DataFrame([kvah_loss_thd, kvah_loss_pf, Vimb_per, Iimb_per]).T
        df_final['kvah_loss_total'] = df_final['kvah_loss_thd'] + df_final['kvah_loss_pf']
        df_final = df_final.fillna(0)
        df = df.fillna(0)
        return df, df_final

In [6]:
cat = PowerFactorLoss_KPI(0.1)
df2, df_final  = cat.output(df)

In [7]:
df2.head()

,DeviceID,cat,EM_Power Factor,EM_Active Power (kW),mean_volt,Mean_THD,EM_THD Voltage,EM_Voltage Ph1-Ph2 (V),EM_Voltage Ph2-Ph3 (V),EM_Voltage Ph1-Ph3 (V),...,PF_S1,kvah_loss_pf,Vmax,Vmin,voltage_imb,Vimb_per,Imax,Imin,current_imb,Iimb_per
0,EM1,4,0.9,1298.8,422.500000,3.733333,1.9,423.5,423.1,420.9,...,1311.919192,0.0,423.5,420.9,1.600000,0.378698,1819.4,1793.6,17.066667,0.942563
1,EM1,14,0.9,1279.5,422.500000,3.733333,1.8,423.7,423.1,420.7,...,1292.424242,0.0,423.7,420.7,1.800000,0.426036,1813.5,1768.8,28.800000,1.613717
2,EM1,21,0.9,1326.1,422.300000,3.900000,1.8,423.4,422.9,420.6,...,1339.494949,0.0,423.4,420.6,1.700000,0.402557,1859.1,1834.6,14.033333,0.759119
3,EM1,35,0.9,1359.2,421.966667,3.700000,2.0,423.0,422.6,420.3,...,1372.929293,0.0,423.0,420.3,1.666667,0.394976,1906.0,1878.3,17.166667,0.905670
4,EM1,49,0.9,1345.8,421.966667,0.000000,0.0,423.0,422.5,420.4,...,1359.393939,0.0,423.0,420.4,1.566667,0.371277,1898.5,1847.5,29.533333,1.573405


In [8]:
df_final

,kvah_loss_thd,kvah_loss_pf,Vimb_per,Iimb_per,kvah_loss_total
DeviceID,,,,,
EM1,25757.731559,2659.369038,0.395883,1.175472,28417.100597
EM10,0.000000,260107.456234,0.319832,8.850954,260107.456234
EM22,0.000000,6896.885207,0.278932,10.525454,6896.885207
EM23,0.000000,-11390.133888,0.204721,8.344041,-11390.133888
EM4,4387.016462,435.847453,0.191162,2.009162,4822.863915
EM5,4675.324937,512.770403,0.061006,6.389125,5188.095341
EM6,0.000000,8536.464134,0.246610,2.133400,8536.464134
EM7,0.000000,0.000000,0.052978,0.000000,0.000000
EM9,0.000000,293.050344,0.198931,1.299309,293.050344
